# 程序说明
时间：2016年11月22日

说明：finetune InceptionV3网络。


## 1.加载keras模块

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


#### 如需绘制模型请加载plot

In [2]:
from keras.utils.visualize_util import plot

In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

86695936/86916664 [============================>.] - ETA: 0s

In [4]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, None, None, 32)896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, None, None, 32)64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, None, None, 32)9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [5]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)


tf


In [12]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [11]:
# train the model on the new data for a few epochs
model.fit_generator(...)

SyntaxError: invalid syntax (<ipython-input-11-af00eea935de>, line 2)

In [13]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

(0, 'input_1')
(1, 'convolution2d_1')
(2, 'batchnormalization_1')
(3, 'convolution2d_2')
(4, 'batchnormalization_2')
(5, 'convolution2d_3')
(6, 'batchnormalization_3')
(7, 'maxpooling2d_1')
(8, 'convolution2d_4')
(9, 'batchnormalization_4')
(10, 'convolution2d_5')
(11, 'batchnormalization_5')
(12, 'maxpooling2d_2')
(13, 'convolution2d_9')
(14, 'batchnormalization_9')
(15, 'convolution2d_7')
(16, 'convolution2d_10')
(17, 'batchnormalization_7')
(18, 'batchnormalization_10')
(19, 'averagepooling2d_1')
(20, 'convolution2d_6')
(21, 'convolution2d_8')
(22, 'convolution2d_11')
(23, 'convolution2d_12')
(24, 'batchnormalization_6')
(25, 'batchnormalization_8')
(26, 'batchnormalization_11')
(27, 'batchnormalization_12')
(28, 'mixed0')
(29, 'convolution2d_16')
(30, 'batchnormalization_16')
(31, 'convolution2d_14')
(32, 'convolution2d_17')
(33, 'batchnormalization_14')
(34, 'batchnormalization_17')
(35, 'averagepooling2d_2')
(36, 'convolution2d_13')
(37, 'convolution2d_15')
(38, 'convolution2d_18

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)